# Learning on your own and project

## Outline

* Learning to read existing code

* Official Haskell documentation

* Using Hoogle and Hackage
  - Hoogle

  - Hackage

  - Researching functions and types from the Data.Map module
  
  - Looking up source code

* Using the Haskell Wiki

* Using **:type** and **:info** in GHCi

* Walkthrough of creating a project

In this lesson, we will learn how you can explore types, functions and modules in Haskell on your own.

As we cover more and more, it is becoming clear that since Haskell is a functional programming language, it is all about functions and their composition.

When designing a Haskell program, we keep this in mind and cut up our code in these small chunks of functions. 

This allows us to reason about smaller portions of the code and their correctness. And, as a bonus, it keeps our code composable, so we can reuse it later.

So, it comes as no surprise that a lot of code is already written by others! It is our task to find useful functions and figure out how to use them.

## Learning to read existing code

Bellow is a small program which we will use to explain how to proceed when you are confronted with code which you do not fully understand.

The general procedure we will follow will be:
- understand the language pragmas 

- understand the functions and types from the import statements

- understand the functions used in the haskell code that do not need import statements

- reason about what the code is doing 

In [ ]:
{-# LANGUAGE DuplicateRecordFields #-}

import Data.Map as Map ( fromList, insert, toList, Map, delete )

data Book = Book 
              { author :: String
              , title :: String
              } deriving Show

data BookWithYear = BookWithYear 
              { author :: String
              , title :: String
              , year :: Int
              } deriving Show

data BookWithPrice = BookWithPrice 
              { author :: String
              , title :: String
              , price :: Double
              } deriving Show

data BookTypes = Year BookWithYear | Price BookWithPrice

book1 :: BookWithYear
book1 = BookWithYear { author = "Douglas Adams"
                     , title = "The Hitchhiker's Guide to the Galaxy"
                     , year = 1985
             }

book2 :: BookWithYear
book2 = BookWithYear { author = "J. R. R. Tolkien"
                     , title = "Lord of the Rings"
                     , year = 1954
             }

book3 :: BookWithPrice
book3 = BookWithPrice { author = "Frank Herbert"
                      , title = "Dune"
                      , price = 18.99
             }

createBookType :: BookTypes -> Book
createBookType (Year (BookWithYear authorData titleData _)) = Book { author = authorData
                                                                    , title = titleData }
createBookType (Price (BookWithPrice authorData titleData _)) = Book { author = authorData
                                                                    , title = titleData }

variousBooks :: [BookTypes]
variousBooks = [Year book1, Year book2]

library :: Map.Map Int Book
library = Map.fromList $ zip [1,2] $ map createBookType variousBooks

listBooks :: Map.Map Int Book -> IO ()
listBooks lib = do
    let books = Prelude.map snd (Map.toList lib)
    print books

addBook :: Map.Map Int Book -> Book -> Map.Map Int Book
addBook lib book = Map.insert (getFreeSpot 1) book lib
    where libKeys = Prelude.map fst $ Map.toList lib
          getFreeSpot spot = if spot `elem` libKeys
                             then getFreeSpot (spot + 1)
                             else spot

removeBook :: Map.Map Int Book -> Int -> Map.Map Int Book
removeBook lib bookID = Map.delete bookID lib

main :: IO ()
main = do
    putStr "Current books in library are:\n"
    listBooks library
    putStr "\nAdding a book to library:\n"
    let library1 = addBook library $ createBookType $ Price book3
    listBooks library1
    putStr "\nRemoving books with bookID = 1 and bookID = 2 from updated library:\n"
    listBooks $ foldr (flip removeBook) library1 [1,2] 

main

First let's list the functions, types and pragmas that we have to know in order to understand the code:
- the *DuplicateRecordFields* language pragma

- the functions `fromList`, `insert`, `toList`, `delete` and the `Map` type from the **Data.Map** module

- the functions `map`, `zip`, `snd`, `fst`, `elem`, `print`, `putStr`, `flip` and `foldr` from **Prelude** module.

<div class="alert alert-block alert-info">
For some IDEs and text editors, it is possible to add the *Haskell language server* extension which automatically parses the code and suggest changing the import statement of a module so that it lists all the used functions from the code: https://wiki.haskell.org/IDEs#Haskell_Language_Server. This makes it easier for the developer to know which functions are used from which module. VSCode is one of such editors.
</div>

Let's pretend that we don't know what the language pragma and the functions from the **Data.Map** module do.

For the **Prelude** functions we will only showcase how to lookup the `foldr` function with the Haskell tools we have.

In the next chapters we will helps ourselvse to understand the code above with the: 
- official Haskell documentation

- web-tools Hoogle and Hackage 

- Haskell wiki page

## Official Haskell documentation

The official Haskell documentation can be found at [https://downloads.haskell.org/ghc/latest/docs/users_guide/](https://downloads.haskell.org/ghc/latest/docs/users_guide/).

When we lookup functions and types we ussually use *Hoogle* and *Hackage* but for language pragmas the documentation page is more appropritate.

In the upper left corner there is a search bar where you can type in the name of the language pragma *DuplicateRecordFields*.

You get the following explanation *Allow definition of record types with identically-named fields.*.

If you want you can also help yourself by looking at the code example the documentation offers. 

In the end you realize that this language pragma enables definition of multiple record syntax types that have same record field names. 

If we check our code we see that the types `Book`, `BookWithYear` and `BookWithPrice` have the `author` and `title` field names the same.

## Using Hoogle and Hackage

### Hoogle

**Hoogle** ([*hoogle.haskell.org*](hoogle.haskell.org)) is a Haskell API search engine, which allows you to search the Haskell libraries on Haskells central package repository (called Hackage), by either function name, or by approximate type signature. 

This tool is an extension to `:info` and `:type` GHCi commands, because with Hoogle you get a more detailed and graphically friendly explanation of types and functions. 

Hoogle also enables you to search for modules contained in Haskell and see information about the functions that are defined in a specific module. 

Let's try it out. We go to the Hoogle webpage and type in our first function `fromList` from the **Data.Map** module. 

We see we get many results for this name and the variour `fromList` functions have also different type signatures.

Below the type signatures is first stated the package name and then its modules that containt the function with the type signature above.

If we want to look up only the `fromList` function in the **Data.Map** module we type into hoogle `fromList +Data.Map`.

Because multiple packages contain this module there are again multiple result. Ussually we are interested in the first one which is the most commonly used.

<img src="../images/lesson15/hoogle1.png"
  style="
  display:block;
  margin-left: 20%;
  margin-right: auto;
  width: 64%;"/>

We see the first result says:
```haskell
fromList :: Ord k => [(k, a)] -> Map k a
```

Below it we see the package name and the modules that contain this definition
```
containers Data.Map.Internal Data.Map.Lazy Data.Map.Strict Data.Map.Strict.Internal
```

If we click on the function name with the type signature we get re-directed to the Hackage page for the first module listed bellow the function.

You could also click on the module names and get redirected to the Hackage page lists the function and type names that the module contains. 

We will cover the Hackage tool shortly. Before we got there we explain some advance search options that Hoogle gives us.

Once we learn also how Hackage works we will cover all the other functions from our code that we are curious about how they work.

Here are some explanations of how hoogle search works:

* **map** searches as text, finding map, concatMap, mapM

* **con map** searches for the text "map" and "con" finding concatMap, but not map

* **a -> a** searches by type, finding id :: a -> a

* **a** searches for the text "a"

* **:: a** searches for the type "a"

* **id :: a -> a** searches for the text "id" and the type "a -> a"

Additional to your search, you can add restriction on the modules that the function resides in. For example, if we search for

* **file -System** this excludes results from modules such as **System.IO**, **System.FilePath.Windows** and **Distribution.System**

* **fromList +Data.Map** finds results in the **Data.Map** module

### Hackage

**Hackage** ([*hackage.haskell.org*](hackage.haskell.org)) is the Haskell community's central package archive of open-source software. 

If you click on any result Hoogle returns, you will be redirected to the Hackage page of it, that shows you information about this function, type, or module.

If we now click on the first result for our `fromList` function search we get redirected to the Hackage page for the **Data.Map.Internal** module.

The page does not load at the beginning but at the place where the `fromList` function is defined.

We see again the type signature from the function and below is an explanation that says:
```haskell
fromList :: Ord k => [(k, a)] -> Map k a 
```
```
Build a map from a list of key/value pairs. If the list contains more than one value for the same key, the last value for the key is retained.
```

There is also a code example added to the explanation to showcase how the function behaves.
```haskell
fromList [] == empty
fromList [(5,"a"), (3,"b"), (5, "c")] == fromList [(5,"c"), (3,"b")]
fromList [(5,"c"), (3,"b"), (5, "a")] == fromList [(5,"a"), (3,"b")]
```

We understand now how we can build a Map object. We provide it with a list of tuples where the first elements in the tuples represents the keys of the Map. They have to have an instance of the `ord` type class. And the second elements represent the values of the Map.

Let us now also explain how a search for a module works. We type into hoogle the search `Data.Map`.

The first result we get in hoogle redirects us to the **Data.Map** hackage page of the **containers** package.

On the beginning of this page, we see the following note:
```
Note: You should use **Data.Map.Strict** instead of this module if:

- You will eventually need all the values stored.
- The stored values don't represent large virtual data structures to be lazily computed.
```

When we click on **Data.Map.Strict**, a new page gets loaded, and we see that this module contains an updated description of all the functions and data typed that are part of the **Data.Map** module. 

Because in the dot notation of modules, we go one step into the tree when you import **Data.Map** everything from the child branches contained in **Data.Map** is also imported. 

Another thing to mention about Hackage is that, when you are looking at a module as **Data.Map.Strict** you may see on the left side a table of contents that defines the module. 

Also in the top-left corner of the page, you can see the name of the package that contains this module. For the module **Data.Map.Strict** the package name is **containers** with current version **0.6.6**. 

If you go to the starting page of Hackage, click on Browse packages you can type in **containers** in the search bar and list of packages that contains this name will be displayed. 

Then you can further click on the package **containers**, and you will get some information about the package together with a list of all the modules it contains, of which **Data.Map** is just one.

<img src="../images/lesson15/hackage1.png"
  style="
  display:block;
  margin-left: 20%;
  margin-right: auto;
  width: 64%;"/>

<div class="alert alert-block alert-info">
If you want to use a module you learned about in your future code, it is worth to check the main page of the module for comments. For example: for the `Data.Map` module we need a qualified import to make sure function do not clash from the `Prelude` module.
</div>

### Researching functions and types from the Data.Map module

Let us got now through all the othe functions and the `Map` type that are defined in the **Data.Map** module. 

Let's first search in hoogle for the `Map` data type: `Map +Data.Map`. The first result in for the **Data.Map** module.

The second result defines the type signature of the `Map` type. If we click on it we get the following definition on the hackage page:
```haskell
data Map k a 
```
```
A Map from keys k to values a.
```

In our case when we define the `library` variable which is a Map, we use `Int` for keys and the `Book` type for values. 

Next we have the function `listBooks` that takes in a Map which holds books and performs some IO actions. We see there we use the function `toList` from the Map module.

From Hoogle we get:
```haskell
toList :: Map k a -> [(k, a)] 
```
```
Convert the map to a list of key/value pairs
```

It takes in a Map object and returns a list of tuples. The opposite of the `fromList` function.

So, the `listBooks` function can take in our library and list all the books that are contained in the Map.

Then we come to the function `addBook` that takes in a Map which contains books, a book, and returns a Map which contains books.

The only function we do not know yet in this code is the `insert` function from the Map module.

From Hoogle we get:
```haskell
insert :: Ord k => k -> a -> Map k a -> Map k a 
```
```
Insert a new key and value in the map. If the key is already present in the map, the associated value is replaced with the supplied value.
```

From the code we see, this function can update our library with a new Book, where it chooses the first free key that the library does not contain yet.

The last function that works with a Map is the `removeBook` function that takes in a Map of Books and an Int, and returns a Map of Books.

We see that it uses the `delete` function from the Map module.

From Hoogle we get:
```haskell
delete :: Ord k => k -> Map k a -> Map k a 
```
```
Delete a key and its value from the map. When the key is not a member of the map, the original map is returned.
```

So this function can delete a book from our library where we identify it with the id of the book, that are the keys in the Map.

In the end we have the main function that lists the books from our library, then adds one book and lists the updated library. And in the end removes the first two books from the updated library and lists the library again. 

In the next chapter we will cover the `foldl` and showcase how to lookup its source code.

### Looking up source code

Let's preted for a moment we do not know how the function `foldr` works.

We type it into hoogle and see that the first two results give us the following type signatures:
```haskell
foldr :: Foldable t => (a -> b -> b) -> b -> t a -> b
foldr :: (a -> b -> b) -> b -> [a] -> b 
```

The first `foldr` is a more general definition that says the last input parameter has to have an instance of the `Foldable` type class.

The second `foldr` left is just defines for lists, which do have an instance of the `Foldable` type class. We click on it.

The hackage site explains us how it works but we are still not sure of it. So we decide to look up the source code of this function.

The source code means how this function is defined using the Haskell language. 

To do this we click on the `Source` sign that is on the right side in the same line as the type signature is. 

We get the following definition for the `foldr` function:
```haskell
foldr :: (a -> b -> b) -> b -> [a] -> b 
foldr k z = go
          where
            go []     = z
            go (y:ys) = y `k` go ys
```

Now we can better reason about what this function is doing. 

In practice not all of Haskells source code is so easy to understand because to the code is written such that it is optimized for performance. 

Let's look at the source code for the `delete` functions of the **Data.Map** module.

```haskell
delete :: Ord k => k -> Map k a -> Map k a
delete = go
  where
    go :: Ord k => k -> Map k a -> Map k a
    go !_ Tip = Tip
    go k t@(Bin _ kx x l r) =
        case compare k kx of
            LT | l' `ptrEq` l -> t
               | otherwise -> balanceR kx x l' r
               where !l' = go k l
            GT | r' `ptrEq` r -> t
               | otherwise -> balanceL kx x l r'
               where !r' = go k r
            EQ -> glue l r
```

We see this source code contains also code which we have not cover yet. So you would need to reserach what does the `prtEq`, `balanceR` and `balanceL` functions do.

## Using the Haskell Wiki

Sometimes it makes also sense look up a function or type on the haskell wiki.

There you can find a more user-friendly explanation of types and classes with more code examples.

But it does not cover everything as the Hackage page does since it is the central repository for Haskell packages.

The Haskell wiki page can be found at [*wiki.haskell.org*](wiki.haskell.org). 

It contains numerous useful articles and links that cover various areas of Haskell, such as:

* Learning Haskell

* Using Haskell

* Joining the community

You can also use the wiki search bar in the top-right corner to search for articles on the Haskell wiki page. 

Keep in mind that for a specif problem it is maybe good to do a Google search, but for learning general things about the Haskell ecosystem this is a good starting point. 

A more up-to-date list of community pages and learning resources can be found on [*haskell.org*](haskell.org):

* https://www.haskell.org/community/

* https://www.haskell.org/documentation/ 

Nevertheless, you could also find interesting educational content if you search on Google or YouTube.

A good example of learning resources that are not stated anywhere on the Haskell wiki page or the Haskell documentation page at current time of writing are: 
- the Stack Overflow tutorial https://devtut.github.io/haskell/ that provides a basic and advanced Haskell tutorial 

- the advance Haskell libraries and tutorial explanations from FP complete project: https://www.fpcomplete.com/haskell/learn/ 

## Using **:type** and **:info** in GHCi

<div class="alert alert-block alert-info">
Remember, GHCi is the <b>interactive</b> shell for GHC (The Glasgow Haskell Compiler). It lets you play and execute code on the go! In this Jupyter notebook, each cell is actually a GHCi shell that we can use for small Haskell snippets. You can also open a GHCi shell in a terminal by executing `ghci` (you can quit by execution <code>:q</code>)
</div>

Two useful GHCi commands, which we extensively used in our lessons before, are `:type` and `:info`. Which also can be abbreviated as `:t` and `:i` in its use. 

These two let us inspect and gather information on types, function and instances in a quick way compared to using Hoogle and Hackage.

To highlight their importance, we consider an example. Imagine that we want to open a file in Haskell, and we remember the existence of a function `openFile` from the `System.IO` module, but not its use. We can use GHCi together with `:t` and `:i` to figure out its usage!

In [ ]:
import System.IO

:t openFile
-- openFile :: FilePath -> IOMode -> IO Handle

:i FilePath
-- type FilePath = String

:i IOMode
-- data IOMode = ReadMode | WriteMode | AppendMode | ReadWriteMode

From this information, we see that the function `openFile` takes in a file path, which is a string. We also see that it takes a mode, from this context we deduce that these are for indicating the file system in what way we are opening this file.

Besides the above deconstruction of the algebraic data type `IOMode`, the `:i` command is also useful for exploring other Haskell types.

For instance, if we type `:i Int` we get additional information about this type. We can even see to which type classes the `Int` type belongs!

```haskell
:i Int
type Int :: *
data Int = GHC.Types.I# GHC.Prim.Int#
  	-- Defined in ‘GHC.Types’
instance Eq Int -- Defined in ‘GHC.Classes’
instance Ord Int -- Defined in ‘GHC.Classes’
instance Enum Int -- Defined in ‘GHC.Enum’
instance Num Int -- Defined in ‘GHC.Num’
instance Real Int -- Defined in ‘GHC.Real’
instance Show Int -- Defined in ‘GHC.Show’
instance Read Int -- Defined in ‘GHC.Read’
instance Bounded Int -- Defined in ‘GHC.Enum’
instance Integral Int -- Defined in ‘GHC.Real’
```

We see that `Int` has many type class instances. We can also look up how one of those type classes is defined.

Let us pick for example the `Num` type class. 

```haskell
:i Num
type Num :: * -> Constraint
class Num a where
  (+) :: a -> a -> a
  (-) :: a -> a -> a
  (*) :: a -> a -> a
  negate :: a -> a
  abs :: a -> a
  signum :: a -> a
  fromInteger :: Integer -> a
  {-# MINIMAL (+), (*), abs, signum, fromInteger, (negate | (-)) #-}
  	-- Defined in ‘GHC.Num’
instance Num Word -- Defined in ‘GHC.Num’
instance Num Integer -- Defined in ‘GHC.Num’
instance Num Int -- Defined in ‘GHC.Num’
instance Num Float -- Defined in ‘GHC.Float’
instance Num Double -- Defined in ‘GHC.Float’
```

We get all the class functions and their type signatures. From the `{-# MINIMAL ... -#}` description, we see which functions are minimally required to implement when creating a Num instance for a user defined type.

Conversely, this also means that any type that is an `Int`, is a member of the `Num` type class, so it has all of the above class functions available. And indeed we can negate the following

In [ ]:
numberOne :: Int
numberOne = 1
:t numberOne
negate numberOne

Lastly, at the end, all built in Haskell types that are members of this type class are listed and in the comments you can see their module location. 

## Walkthrough of creating a project

In this section, we will show the creation of a small Haskell project. We will show code snippets step by step to sketch out the implementation. 

Besides these snippets, we will also give clarification on the design decisions taken along the way. The goal here is to give you a feeling for how to choose the correct types and function when building a project.

The project idea is a program that functions similar to an online web shop. It simulates a shopping basket, and it has a database with items that you can add to the shopping basket. 

Additionally, once the shopping is done, the program can reduce the price of the total shopping basket, depending on a membership program. 

For simplicity, we will only have three categories in this online shop, music, books and coffee.

So, how can we translate this idea to code? We might try to implement the Database structure as a `Map` from the module `Data.Map` described above.


This database `Map` contains three entries, one for each class of items. So, it's either one of the keys `"music"`, `"books"` or `"coffee"`.

```json
{
    "music": ...
    "books": ...
    "coffee": ...
}
```

The shop can have multiple kinds of music, books or coffee that they sell. So, we might choose for each of these categories in our database `Map` to have as their value another `Map`. 

Here, each object in a category will have as its key its index in the category `Map`.

<div class="alert alert-block alert-info">
Alternatively, you might consider a list of items instead of a <code>Map</code>, but note that <code>Map</code> is easier to use as a database since you can access the data via keys.
</div>

So, what do the items in each category's look like? Coffee is described differently than a book in a shop, so each value might have a different type describing these attributes. 

We might consider making a custom type for each. But, to keep it simple, we will only track for each item its name and its price. 

The name would be typed as a `String` and initially, we will type the price as a `Double`. 

We choose a `Double` here because we want to track a more precise price for the items and later have the opportunity to round it accordingly.

```json
{
    "name": "A book about the connection of good coffee and music"
    "price": 10
}
```

<div class="alert alert-block alert-info">
    Notice that a <code>Double</code> has fixed precision and may lead to rounding errors in the program, this will be addressed later.
</div>

Besides the core of our program keeping track of the shopping basket, we would also like a minimal interface which uses IO to operate our program.

The list of action available should be presented when running the program, and we would like to have the following actions:
```
Welcome to the store. Possible actions are:
- list_commands
- list_groups
- list_items  --group
- buy_item  --group  --item_index  --quantity
- list_basket
- remove_item  --group  --item_index  --quantity
- get_price  --membership_scheme
- exit
```

Here we call the text after the `--` a flag. This indicates that an extra argument/option must be given to execute that action. 

Each action should change the current shopping basket given the action and return again to the list of possible actions. 

As mentioned, we wanted a membership program in the store. This is also reflected in the `get_price --membership_scheme` action with its flag. 

We call these three membership schemes: "basic", "gold", "platinum", and members will get the following rates
- (Basic 10% | Gold 20% | Platinum 30% ) off for "music" store items

- (Basic 20% | Gold 30% | Platinum 40% ) off for "books" store items

- (Basic 30% | Gold 40% | Platinum 50% ) off for "coffee" store items

### Implementing the code

First, we create a simple database that contains some store items by writing a string to a CSV file.

In [ ]:
csvDatabase :: String
csvDatabase = "music name \"ABBA greatest hits\",price 80/9\n\
              \music name \"Beatles greatest hits\",price 100/9\n\
              \music name \"Coldplay greatest hits\",price 120/9\n\
              \book name \"English vocabulary\",price 230/9\n\
              \book name \"German vocabulary\",price 240/9\n\
              \book name \"Japanese vocabulary\",price 250/9\n\
              \coffee name \"Cappuccino\",price 13/9\n\
              \coffee name \"Latte macchiato\",price 11/9\n\ 
              \coffee name \"Espresso\",price 12/9"

writeDatabase :: IO ()
writeDatabase = do
    writeFile "./database.csv" csvDatabase

writeDatabase

Then we write a program that reads out this database and writes in to the `storeDatabase` variable that is of type Map. 

In [ ]:
import Data.Map as Map 
import Data.List.Split (splitOn)

type ItemType = String
type ItemName = String
type ItemPrice = String
type ItemDatabase = Map.Map Int StoreItem

data StoreItem = Item 
                  { name :: String
                  , price :: Double
                  } deriving Show

parseCSVFile :: IO [(ItemType, ItemName, ItemPrice)]
parseCSVFile = do
    fileContent <- readFile "./database.csv"
    let fileContentLines = lines fileContent 
        removeEmpty [] = []
        removeEmpty list@(x:xs) = if x == ""
                                  then removeEmpty xs 
                                  else list
        itemsList = Prelude.map processContents $ removeEmpty fileContentLines
    return itemsList
    where 
        processContents oneLine = case head (words oneLine) of
            "music" -> ("music", extractName oneLine, extractPrice oneLine)
            "book" -> ("book", extractName oneLine, extractPrice oneLine)
            "coffee" -> ("coffee", extractName oneLine, extractPrice oneLine)
            _ -> ("","","")
        extractName oneLine = splitOn "\"" (head $ splitOn "," (splitOn "name" oneLine !! 1)) !! 1
        extractPrice oneLine = tail (splitOn "price" oneLine !! 1)

createStoreItem :: (ItemType, ItemName, ItemPrice) -> (ItemType, StoreItem)
createStoreItem (itemtype, itemName, itemPrice) = 
    (itemtype, Item {name = itemName, price = priceDouble})
    where priceDouble = nums !! 0 / nums !! 1 :: Double
          nums = Prelude.map read (splitOn "/" itemPrice) :: [Double]

createDatabase :: ItemType -> ItemDatabase -> [(ItemType, StoreItem)] -> ItemDatabase 
createDatabase itemType initDatabase [] = initDatabase
createDatabase itemType initDatabase storeItems = 
    createDatabase itemType updatedDatabase (tail storeItems)
    where item = head storeItems
          updatedDatabase = if fst item == itemType 
                            then Map.insert (getFreeSpot 1) (snd item) initDatabase
                            else initDatabase
          libKeys = Prelude.map fst $ Map.toList initDatabase
          getFreeSpot spot = if spot `elem` libKeys
                             then getFreeSpot (spot + 1)
                             else spot

musicIO :: IO ItemDatabase 
musicIO = do
    csvData <- parseCSVFile
    return $ createDatabase "music" initDatabase (Prelude.map createStoreItem csvData)
    where initDatabase =  Map.empty

booksIO :: IO ItemDatabase 
booksIO = do
    csvData <- parseCSVFile
    return $ createDatabase "book" initDatabase (Prelude.map createStoreItem csvData)
    where initDatabase =  Map.empty

coffeeIO :: IO ItemDatabase 
coffeeIO = do
    csvData <- parseCSVFile
    return $ createDatabase "coffee" initDatabase (Prelude.map createStoreItem csvData)
    where initDatabase =  Map.empty

storeDatabase :: IO (Map.Map String ItemDatabase)
storeDatabase = do
    music <- musicIO
    books <- booksIO
    coffee <- coffeeIO
    return $ Map.fromList [("music",music),("books",books),("coffee",coffee)]

Now that we have our database, let's write the code that interacts with this database. 

The entry point of our program is the `main` function that will display the welcome message to the user and print the available commands. 

This function will also introduce an empty list `init_state` that represented the initial empty shopping basket.

The workflow of what follows is the recursively handling of actions that act and update the last state of the shopping basket. 

We name the function that we will recursively call, the `startShoping` function. This function will also parse user input and act on this input. 

In the `startShoping` function, the following function will represent some of the different actions

- `listCommands` (displays possible commands)

- `listGroups` (displays shopping groups)

- `listBasket` (displays the basket items)

- `wrongCommand` (displays an info message to the user)

For now, we do not implement any other actions and just return the Haskell variable `undefined` so we know they still have to be implemented. 

After this, we will implement one by one these undefined functions and explain for each how it works.  

In [ ]:
main :: IO ()
main = do
    putStrLn "Welcome to the store."
    printCommands

    let init_state = []
    startShoping init_state

printCommands :: IO ()
printCommands = do
    putStrLn "Possible actions are:\n \
            \ list_commands \n \
            \ list_groups \n \
            \ list_items  --group \n \
            \ buy_item  --group  --item_index  --quantity \n \
            \ list_basket \n \
            \ remove_item  --group  --item_index  --quantity \n \
            \ get_price  --membership_scheme \n \
            \ exit \n\n \
            \ Example command: list_items music"

type CommandOptions = [String]
type ShoppingGroup = String
type ItemIndex = Int
type ItemQuantity = Int
type ShoppingBasket = [(ShoppingGroup,ItemIndex,ItemQuantity)]

startShoping :: ShoppingBasket -> IO ()
startShoping basket = do
    putStrLn "\nPlease select what you want to do: "
    fullCommand <- getLine
    let (command:options) = words fullCommand

    if command == "exit"
    then putStrLn "Exiting shopping."
    else do
        let function = case command of
                "list_commands" -> listCommands
                "list_groups" -> listGroups
                "list_items" -> listItems
                "buy_item" -> buyItem
                "list_basket" -> listBasket
                "remove_item" -> removeItem
                "get_price" -> getPrice
                _ -> wrongCommand

        updatedBasket <- function basket options
        startShoping updatedBasket

listCommands :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
listCommands basket options = do
    printCommands
    return basket

listGroups :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
listGroups basket options = do
    putStrLn "Shopping groups are: music, books, coffee."
    return basket

listItems :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
listItems basket options = do
    return undefined

buyItem :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
buyItem basket options = do
    return undefined

listBasket :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
listBasket basket options = do
    print basket
    return basket

removeItem :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
removeItem basket options = do
    return undefined

getPrice :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
getPrice basket options = do
    return undefined

wrongCommand :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
wrongCommand basket options = do
    putStrLn "The command you entered is not correct."
    return basket

Now let's first implement the function `listItems` that lists the basket items and their indexes. 

We let the user know if the command option he typed in is incorrect, or if he typed in as many or no options.

We use the `notElem` function that works the same as the `elem` function, just that is says True if the element is not contained in the list.

In [ ]:
import Data.Maybe 

listItems :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
listItems basket options = do
    storeDatabase <- storeDatabaseIO
    case length options of
        1 -> do
            let group = head options
            if group `notElem` ["music","books","coffee"]
            then do
                putStrLn "This group does not exist."
                return basket
            else do
                let itemDatabase itemName = fromJust $ Map.lookup "music" storeDatabase 
                    groupData = case group of
                        "music" -> itemDatabase "music"
                        "books" -> itemDatabase "books" 
                        "coffee" -> itemDatabase "coffee" 
                        _ -> Map.empty
                print (Map.toList groupData)
                return basket
        _ -> do
            putStrLn "To much or to less options for this command."
            return basket 

Next, we implement the function `buyItem` that lets us add an item to the basket.

We use the function `readMaybe` from the **Text.Read** module and some helper functions to work with the `Maybe` type.

The `readMaybe` function takes a string and returns a maybe value. In the case the string can be read as a number, the function returns a Just value of the number, else it returns Nothing.
```haskell
readMaybe :: Read a => String -> Maybe a
```

We make the following checks on the command parameters and notify the user if any fails:
- is the count of the supply command parameters correct

- are the command parameters legal values

We also check if the item we want to add already exists in the basket and update the quantity accordingly.

In [ ]:
import Text.Read

buyItem :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
buyItem basket options 
        | optionsLenghtCheck && groupCheck && itemIndexCheck && quantityCheck = do
            let itemIndexNum = read itemIndex :: Int
                quantityNum = read quantity :: Int
                checkItem = (\(x,y,_) -> x == group && y == itemIndexNum)
                existingItem = Prelude.filter checkItem basket
            if length existingItem == 1
            then do
                let (_,_,existingQuantity) = head existingItem
                    newQuantity = existingQuantity + quantityNum
                    updatedBasket = Prelude.filter (not . checkItem) basket ++ [(group, itemIndexNum, newQuantity)]
                return updatedBasket
            else do
                let updatedBasket = basket ++ [(group, itemIndexNum, quantityNum)]
                return updatedBasket
        | optionsLenghtCheck = do
            putStrLn "One or more of the options have not a legal value."
            return basket
        | otherwise = do
            putStrLn "To much or to less options for this command."
            return basket
    where optionsLenghtCheck = length options == 3
          (group:itemIndex:quantity:_) = options
          groupCheck = group `elem` ["music","books","coffee"]
          itemIndexCheck = let maybeInt = readMaybe itemIndex :: Maybe Int
                               integerCheck = isJust maybeInt
                           in
                               if integerCheck then (fromJust maybeInt) `elem` [1,2,3] else False
          quantityCheck = let maybeInt = readMaybe quantity :: Maybe Int
                              integerCheck = isJust maybeInt
                          in
                              if integerCheck then (fromJust maybeInt > 0) else False

Then we implement the `removeItem` function that lowers the amount of an item in the basket or deletes the item entirely.

We use the same functions and do the same checks as in the previous function `buyItem`.

In [ ]:
removeItem :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
removeItem basket options
        | optionsLenghtCheck && groupCheck && itemIndexCheck && quantityCheck = do
            let itemIndexNum = read itemIndex :: Int
                quantityNum = read quantity :: Int
                checkItem = (\(x,y,_) -> x == group && y == itemIndexNum)
                existingItem = Prelude.filter checkItem basket
            if length existingItem == 1
            then do
                let (_,_,existingQuantity) = head existingItem
                    newQuantity = existingQuantity - quantityNum
                    updatedBasket = let filteredBasket = Prelude.filter (not . checkItem) basket
                                    in
                                        if newQuantity > 0
                                        then filteredBasket ++ [(group, itemIndexNum, newQuantity)]
                                        else filteredBasket
                return updatedBasket
            else do
                putStrLn "Item does not exist in basket."
                return basket
        | optionsLenghtCheck = do
            putStrLn "One or more of the options have not a legal value."
            return basket
        | otherwise = do
            putStrLn "To much or to less options for this command."
            return basket
    where optionsLenghtCheck = length options == 3
          (group:itemIndex:quantity:_) = options
          groupCheck = group `elem` ["music","books","coffee"]
          itemIndexCheck = let maybeInt = readMaybe itemIndex :: Maybe Int
                               integerCheck = isJust maybeInt
                           in
                               if integerCheck then (fromJust maybeInt) `elem` [1,2,3] else False
          quantityCheck = let maybeInt = readMaybe quantity :: Maybe Int
                              integerCheck = isJust maybeInt
                          in
                              if integerCheck then (fromJust maybeInt > 0) else False

<div class="alert alert-block alert-info">
    <b>ADITIONAL EXERCISE:</b> If you look at the code above and compare it to the previous function you see that 4 out of 6 do-blocks are the same. Try to rewrite the above functions in such a way that you extract the common part as much as possible and call it from those functions.
</div>

The last function we implement is `getPrice`, which calculates from our basket the price we have to pay and displays it to the user.

First, we make some checks if the options for the command are correct and let the user know if they are not.

If everything is ok, we call the `printPrice` function which calculates the price of the basket items, taking their quantity and membership discount in account.

In [ ]:
getPrice :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
getPrice basket options 
    | (length options <= 1) && (length options /= 0) = 
        if head options `notElem` ["basic", "gold", "platinum"]
        then do
            putStrLn "Incorrect option for this command."
            return basket
        else do
            printPrice basket options
            return basket
    | length options <= 1 = do
        printPrice basket options
        return basket
    | otherwise = do
        putStrLn "To much options for this command."
        return basket 

printPrice :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
printPrice basket options = do
    storeDatabase <- storeDatabaseIO
    let (membership, discountCoef) = if Prelude.null options
        then ("", const . const 1)
        else (head options, discount)
        calculatePrice (group,itemIndex,quantity) = fromIntegral quantity * 
                                                        discountCoef membership group *
                                                            (price $ fromJust 
                                                                (Map.lookup itemIndex $ fromJust 
                                                                    (Map.lookup group storeDatabase)))
        basketPrice = sum $ Prelude.map calculatePrice basket
    putStrLn $ "Price for your basket is: " ++ show basketPrice
    return basket
  where discount membership grp = let membershipFactor = case membership of
                                                             "basic" -> 0
                                                             "gold" -> 1
                                                             "platinum" -> 2
                                  in case grp of
                                         "music" -> 0.9 - 0.1*membershipFactor
                                         "books" -> 0.8 - 0.1*membershipFactor
                                         "coffee" -> 0.7 - 0.1*membershipFactor

We get some rounding errors because the `Double` type can handle only 16 digits of precision, when the lead decimal number is not a 0.

Let us try to solve this problem by searching for a more accurate type than `Double` and comparing the results.

### Minimizing rounding errors

If we google for *Haskell number types* we find that the built-in number types are `Num, Real, Integer, Int, Ratio, Rational, Double, Float`.

Because we need larger decimal precision than the `Double` type has to offer, we choose the `Rational` type to re-implement our code.

We test this type with the code below, where we can see how this type can help us to preserve precision.

In [ ]:
-- without loosing precision
fromRational ((7/25 :: Rational)*25) :: Double

-- loosing some precision
(fromRational (7/25 :: Rational)) * 25 :: Double

We need to change the type of the price field in the `StoreItem` data type from `Double` to `Rational`. 

We also change the type signature of the variable `priceDouble` in `createStoreItem` function and we re-name it to `priceRational`.

In [ ]:
data StoreItem = Item 
                  { name :: String
                  , price :: Rational
                  } deriving Show

createStoreItem :: (ItemType, ItemName, ItemPrice) -> (ItemType, StoreItem)
createStoreItem (itemtype, itemName, itemPrice) = 
    (itemtype, Item {name = itemName, price = priceRational})
    where priceRational = fromIntegral (nums !! 0) / fromIntegral (nums !! 1)  :: Rational
          nums = Prelude.map read (splitOn "/" itemPrice) :: [Int]

After that, we have to update the `printPrice` function such that it works with `Rational` type numbers and prints the result in `Double`.

In [ ]:
printPrice :: ShoppingBasket -> CommandOptions -> IO ShoppingBasket
printPrice basket options = do
    storeDatabase <- storeDatabaseIO
    let (membership, discountCoef) = if Prelude.null options
        then ("", const . const 1)
        else (head options, discount)
        calculatePrice (group,itemIndex,quantity) = fromIntegral quantity * 
                                                        discountCoef membership group *
                                                            (price $ fromJust 
                                                                (Map.lookup itemIndex $ fromJust 
                                                                    (Map.lookup group storeDatabase)))
        basketPrice = fromRational (sum $ Prelude.map calculatePrice basket) :: Double
    putStrLn $ "Price for your basket is: " ++ show basketPrice
    return basket
  where discount membership grp = let membershipFactor = case membership of
                                                             "basic" -> 0
                                                             "gold" -> 1
                                                             "platinum" -> 2
                                  in case grp of
                                         "music" -> (9 - 1*membershipFactor)/10 :: Rational
                                         "books" -> (8 - 1*membershipFactor)/10 :: Rational
                                         "coffee" -> (7 - 1*membershipFactor)/10 :: Rational

We expect that we will lose some precision in the final rounding, but not in the steps when we calculate the final price by using the `Rational` type.

We define the following case of shopping:
- buy_item music 1 4

- buy_item books 2 5

- buy_item coffee 3 6

The final basket prices for the `Double` and `Rational` types for all memberships are:
- get_price basic
  <br>Price for your basket is: 144.26666666666668   (Double)
  <br>Price for your basket is: 144.26666666666668   (Rational)

- get_price gold
  <br>Price for your basket is: 126.57777777777778   (Double)
  <br>Price for your basket is: 126.57777777777778   (Rational)

- get_price platinum
  <br>Price for your basket is: 108.8888888888889    (Double)
  <br>Price for your basket is: 108.88888888888889   (Rational)

The only case where we profit some precision is for the platinum membership. 

The other two cases give us the same result if in the end convert the `Rational` result to `Double`.

We are still not happy and google for *Haskell rounding error*. We see there is another type called `Decimal`. 

On Hoogle, if we choose the **Data.Decimal** module, we see that the smallest number for a Decimal type is 10^-255.

To make use of this module, you need to install the **Decimal** package using cabal.

The `Decimal` type is, in general, more precise as the `Double` type but also here sometimes rounding errors appear on 255 decimal places, as in the code below.

In [ ]:
import Data.Decimal 

a = (25/7) :: Decimal
print a
b = a * 7
print b

Let's re-write our `StoreItem` data type so that we use the `Decimal` type for the store item prices.

Also for this case we need to update our `createStoreItem` such that it works with the `Decimal` type.

In [ ]:
import Data.Decimal

data StoreItem = Item 
                  { name :: String
                  , price :: Decimal
                  } deriving Show

createStoreItem :: (ItemType, ItemName, ItemPrice) -> (ItemType, StoreItem)
createStoreItem (itemtype, itemName, itemPrice) = 
    (itemtype, Item {name = itemName, price = priceDecimal})
    where priceDecimal = nums !! 0 / nums !! 1 :: Decimal
          nums = Prelude.map read (splitOn "/" itemPrice) :: [Decimal]

Here we do not have to update the `printPrice` function. We can use the same version that we used in the begining for the `Double` type.

We again pick the same shopping case:
- buy_item music 1 4

- buy_item books 2 5

- buy_item coffee 3 6

The results for all three types are as follows:
- get_price basic
  <br>Price for your basket is: 144.26666666666668   (Double)
  <br>Price for your basket is: 144.26666666666668   (Rational)
  <br>Price for your basket is: 144.26666666666666...<254 times 6>...67 (Decimal)

- get_price gold
  <br>Price for your basket is: 126.57777777777778   (Double)
  <br>Price for your basket is: 126.57777777777778   (Rational)
  <br>Price for your basket is: 126.57777777777777...<254 times 7>...78 (Decimal)

- get_price platinum
  <br>Price for your basket is: 108.8888888888889    (Double)
  <br>Price for your basket is: 108.88888888888889   (Rational)
  <br>Price for your basket is: 108.8888888888888...<254 times 8>...89  (Decimal)

We see that the `Decimal` type gives us the most accurate precision for our case. 

In this example, the rounding errors are not that big for a store that has many customers the errors sum up and in the end can make a significant difference.

## Recap

In this lesson, we have discussed:

- how you can help yourself with the `:i` and `:t` commands

- how to help yourself with **Hoogle** and **Hackage**

- we mentioned where to find additional learning resources

- we did a walkthrough of looking at existing code that uses the **Data.Map.Strict** module

- we did a walkthrough of a project where we learned how to minimize rounding errors